# Decision Trees

In this Notebook we're going to explore the development of a decision tree from scratch.

## Background on Decision Trees

## Gini index

The Gini index is the name of the cost function used to evaluate splits in the dataset. Note, you can use other cost functions, but we'll stick with the Gini index.

A Gini score gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5 (for a 2 class problem).

### Example of Gini Index 

Suppose we have two groups of data with 2 rows in each group. The rows in the first group all belong to class 0 and the rows in the second group belong to class 1, so it’s a perfect split.

```python
proportion = count(class_value) / count(rows)
```

The proportions for this example would be:


```python
group_1_class_0 = 2 / 2 = 1
group_1_class_1 = 0 / 2 = 0
group_2_class_0 = 0 / 2 = 0
group_2_class_1 = 2 / 2 = 1
```
Gini is then calculated for each child node as follows:



```python
gini_index = sum(proportion * (1.0 - proportion))
gini_index = 1.0 - sum(proportion * proportion)
```

The Gini index for each group must then be weighted by the size of the group, relative to all of the samples in the parent, e.g. all samples that are currently being grouped. We can add this weighting to the Gini calculation for a group as follows:



```python
gini_index = (1.0 - sum(proportion * proportion)) * (group_size/total_samples)
```
In this example the Gini scores for each group are calculated as follows:



```python
Gini(group_1) = (1 - (1*1 + 0*0)) * 2/4
Gini(group_1) = 0.0 * 0.5 
Gini(group_1) = 0.0 
Gini(group_2) = (1 - (0*0 + 1*1)) * 2/4
Gini(group_2) = 0.0 * 0.5 
Gini(group_2) = 0.0
```

The scores are then added across each child node at the split point to give a final Gini score for the split point that can be compared to other candidate split points.

**The Gini for this split point would then be calculated as 0.0 + 0.0 or a perfect Gini score of 0.0.**

In [65]:
def gini_index(groups, classes):
	# count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    print('Number of Instances {}'.format(n_instances))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        print('group {}'.format(group))
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            #for last element in list of each entry in the group, what is the count of the class matching the classes
            p = [row[-1] for row in group].count(class_val) / size
            print('{} Class p {}'.format(class_val, p))
            score += p * p
        # weight the group score by its relative size of the group!
        gini += (1.0 - score) * (size / n_instances)
        print('Gini {}'.format(gini))
    print('Final Gini {}'.format(gini))

    return gini
 
# test Gini values
print(gini_index([
    [
        [None, 1], [None, 1] #left
    ],
    [
        [None, 1], [None, 1], [None,1]
    ]], 
    [0, 1]))

# print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))



Number of Instances 5.0
group [[None, 1], [None, 1]]
0 Class p 0.0
1 Class p 1.0
Gini 0.0
group [[None, 1], [None, 1], [None, 1]]
0 Class p 0.0
1 Class p 1.0
Gini 0.0
Final Gini 0.0
0.0


## Understand Splitting of Data Using Gini Index

Let's take a look at splitting the data

 - With the Gini function above and the test split function we now have everything we need to evaluate splits.

 - Given a dataset, we must check every value on each attribute as a candidate split, evaluate the cost of the split and find the best possible split we could make.

 - Once the best split is found, we can use it as a node in our decision tree.

 - a dictionary will be used to represent a node in the decision tree as we can store data by name. When selecting the best split and using it as a new node for the tree we will store the index of the chosen attribute, the value of that attribute by which to split and the two groups of data split by the chosen split point.




X1, 		X2,			Y <br>
2.771244718,		1.784783929,		0


In [54]:
# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            print('TESTING SPLIT L {} R {}'.format(len(groups[0]),len(groups[1])))
            gini = gini_index(groups, class_values)
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [55]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [56]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,0]]

split = get_split(dataset)

print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

TESTING SPLIT L 1 R 9
Number of Instances 10.0
group [[1.728571309, 1.169761413, 0]]
0 Class p 1.0
1 Class p 0.0
2.209014212 Class p 0.0
Gini 0.0
group [[2.771244718, 1.784783929, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 0]]
0 Class p 0.4444444444444444
1 Class p 0.4444444444444444
2.209014212 Class p 0.1111111111111111
Gini 0.5333333333333333
Final Gini 0.5333333333333333
X1 < 2.771 Gini=0.533
TESTING SPLIT L 0 R 10
Number of Instances 10.0
group []
group [[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 0]]
0 Class p 0.5
1 Class p 0.4
2.209

### Terminal Nodes

We need to decide when to stop growing a tree.

We can do that using the depth and the number of rows that the node is responsible for in the training dataset.

 - Maximum Tree Depth. This is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop splitting adding new nodes. Deeper trees are more complex and are more likely to overfit the training data.
 - Minimum Node Records. This is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training patterns are expected to be too specific and are likely to overfit the training data.



There is one more condition. It is possible to choose a split in which all rows belong to one group. In this case, we will be unable to continue splitting and adding child nodes as we will have no records to split on one side or another.

Now we have some ideas of when to stop growing the tree. When we do stop growing at a given point, that node is called a terminal node and is used to make a final prediction.

This is done by taking the group of rows assigned to that node and selecting the most common class value in the group. This will be used to make predictions.

Below is a function named to_terminal() that will select a class value for a group of rows. It returns the most common output value in a list of rows.

In [69]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

### Recursive Splitting

We know how and when to create terminal nodes, now we can build our tree.

Building a decision tree involves calling the above developed get_split() function over and over again on the groups created for each node.

New nodes added to an existing node are called child nodes. A node may have zero children (a terminal node), one child (one side makes a prediction directly) or two child nodes. We will refer to the child nodes as left and right in the dictionary representation of a given node.

Once a node is created, we can create child nodes recursively on each group of data from the split by calling the same function again.

Below is a function that implements this recursive procedure. It takes a node as an argument as well as the maximum depth, minimum number of patterns in a node and the current depth of a node.

You can imagine how this might be first called passing in the root node and the depth of 1. This function is best explained in steps:

 - Firstly, the two groups of data split by the node are extracted for use and deleted from the node. As we work on these groups the node no longer requires access to these data.
 - Next, we check if either left or right group of rows is empty and if so we create a terminal node using what records we do have.
 - We then check if we have reached our maximum depth and if so we create a terminal node.
 - We then process the left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
 - The right side is then processed in the same manner, as we rise back up the constructed tree to the root.

In [85]:

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Print a decision tree
def print_tree(node, depth=3):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [95]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

tree = build_tree(dataset, 1, 1)


TESTING SPLIT L 1 R 9
Number of Instances 10.0
group [[1.728571309, 1.169761413, 0]]
0 Class p 1.0
1 Class p 0.0
Gini 0.0
group [[2.771244718, 1.784783929, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]]
0 Class p 0.4444444444444444
1 Class p 0.5555555555555556
Gini 0.4444444444444444
Final Gini 0.4444444444444444
X1 < 2.771 Gini=0.444
TESTING SPLIT L 0 R 10
Number of Instances 10.0
group []
group [[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]]
0 Class p 0.5
1 Class p 0.5
Gini 0.5
Final Gini 0.5
X1 < 1.729 Gini=0.500
TESTING SPLIT L 

In [96]:
print_tree(tree)

   [X1 < 6.642]
    [0]
    [1]


In [97]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [100]:
stump = {'index': 0, 'right': 1, 'value': 6.612287351, 'left': 0}
for row in dataset:
	prediction = predict(stump, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
